##### 필요 모듈 설치

In [2]:
!pip install pandas

In [129]:
!pip install openpyxl

     -------------------------------------- 242.1/242.1 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install requests

In [28]:
!pip install speechrecognition 

     ---------------------------------------- 32.8/32.8 MB 4.1 MB/s eta 0:00:00


In [30]:
!pip install gTTS

     ---------------------------------------- 96.6/96.6 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4


In [32]:
!pip install playsound

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=d3148bb9d90ba3738499592679c86b0a4e1cff3e4f5345e6d602e172e3f15877
  Stored in directory: c:\users\hhjoo\appdata\local\pip\cache\wheels\6a\77\c4\6ed630d7b0e68442ed69150ebac62c3220a835b5001ab4cf7c
Successfully built playsound


In [71]:
!pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 8.8 MB/s eta 0:00:00


##### 라이브러리 불러오기

In [1]:
#오디오 전처리
import librosa #wav 파일을 불러와 파형을 가공하여 fft()나 mfcc등 다양항 형태로 변환가능
#소리는 기본적으로 특정 주파수를 가지는 sin함수들의 합입니다.
#특정 시간에 주파수 성분이 어떻게 구성되어 있는지 확인할 수 있는데요.
#음성 데이터 분석을 할 때 주파수 분석 기법을 많이 사용합니다. (파형 자체를 이용하기도 합니다!)
import librosa.display as dsp
from IPython.display import Audio

#데이터 전처리
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm 

#형태소 분석
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Okt
import json #json파일에 있는 대사 txt불러오기
import re #대사 텍스트 내 단어 수정

#테스트보이스 텍스트화
import speech_recognition as sr

#벡터간 거리 계산
import scipy as sp 

#sklearn이 제공하는 문장을 벡터로 변환하는 함수
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#텍스트 음성출력
from gtts import gTTS
import playsound

#학습시킬 모델
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import train_test_split

#정확도 출력
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier


##### 함수

In [2]:
#테스트 음성 텍스트화 후 벡터값 비교
def dist_raw(v1,v2):
    delta= v1-v2
    return sp.linalg.norm(delta.toarray())
    #각각의 제곱을 더하고 루트를 씌운다

# 형태소 분석 후 말뭉치 만들기
def word_tokens(sentence_list):
    from konlpy.tag import Okt
    t=Okt()
    new_word_tokens=[t.morphs(row) for row in sentence_list]
    new_word_vectorize=[]

    for content in new_word_tokens:
        sentence=''
        for word in content:
            sentence=sentence+' '+word
        new_word_vectorize.append(sentence)
    return new_word_vectorize

# 유클리드거리 Normalize 후 거리 계산
def dist_norm(v1,v2):
        import scipy as sp
        v1_normalized=v1/sp.linalg.norm(v1.toarray())
        v2_normalized=v2/sp.linalg.norm(v2.toarray())
        delta = v1_normalized - v2_normalized

        return sp.linalg.norm(delta.toarray())

##### 음성데이터 dataset

In [4]:
#아날로그 데이터로 되어있는 음성 데이터를 디지털 신호로 변환
#librosa 라이브러리를 사용해 음성 데이터를 load

#예시로 음성데이터 load
elsound, sr= librosa.load('./노인남여_노인대화59_M_ilikepeng_60_전라_실내/노인남여_노인대화59_M_ilikepeng_60_전라_실내_64451.wav', sr=16000)
print('sample_rate:', sr, ',audio shape:', elsound.shape)
print('length:', elsound.shape[0]/float(sr),'secs')
#sampling rate의 의미는 초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미입니다.
#(1초에 음성 신호를 16000번 sampling)
#16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다.

sample_rate: 16000 ,audio shape: (189219,)
length: 11.8261875 secs


In [3]:
#음성데이터로 데이터프레임 생성하기.

#학습시킬데이터
def train_dataset():
    folder= "./el_voice/"
    dataset= []
    for file in tqdm(os.listdir(folder),colour='green'): #tqdm은 진행progress보여주는 함수, os는 디렉토리불러오기
        if 'wav' in file:
            voice_file_path= os.path.join(folder,file) #os.path.join 폴더와 파일 경로 합치기
            data,sr= librosa.load(voice_file_path,sr=16000) #파일별로 하나씩 librosa로 load
            f_name= str(file) #지역명을 파일이름에서 뽑아내기위해 파일명 string으로 변환
            if "수도권" in str(file): #만약 파일이름에 수도권이 있다면
                 soodo_label= "수도권" #수도권 라벨이라 생성하고
                 dataset.append([data,soodo_label]) #데이터셋에 넣기
                 
            else:   #이외에
                gibang_label= f_name[str(file).find('_실')-2:str(file).find('_실')] #파일이름의 (_실) 앞 2글자부터 (_실) -1까지
                #if "도권" in gibang_label: #하지만 (도권)이 들어가있다면 = 이미 if절에서 수도권빠짐
                    #gibang_label.remove("도권") #(도권)은 빼라 
                dataset.append([data,gibang_label]) #데이터셋에 넣기
            
            

    print("dataset완성")
    return pd.DataFrame(dataset,columns=['data','label'])
    #if region_label= '수도권':


#테스트데이터
def test_dataset():
    folder= "./input/"
    dataset_1= []
    for file in tqdm(os.listdir(folder),colour='green'): #tqdm은 진행progress보여주는 함수, os는 디렉토리불러오기
        if 'wav' in file:
            input_file_path= os.path.join(folder,file) #os.path.join 폴더와 파일 경로 합치기
            data,sr= librosa.load(input_file_path,sr=16000) #파일별로 하나씩 librosa로 load
            dataset_1.append([data,file]) #데이터셋에 넣기
                 
            
            
            

    print("dataset완성")
    return pd.DataFrame(dataset_1,columns=['data','file_name'])
    

In [4]:
voice_wav= train_dataset() #이 데이터셋을 voice_wav라 부르겠다

100%|██████████| 15914/15914 [00:29<00:00, 531.16it/s] 


dataset완성


In [5]:
voice_wav

,data,label
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",수도권
2,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권
3,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권
4,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권
...,...,...
7952,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
7953,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
7954,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라
7955,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라


In [6]:
voice_wav['label'].unique() #라벨에 들어간 종류

array(['수도권', '전라', '제주', '충청', '경상'], dtype=object)

In [7]:
#지역마다 번호 나타내는 컬럼 추가


voice_wav['code']= voice_wav['label'].copy() 
#새로운칼럼 생성 라벨 컬럼복제


#실험
#if voice_wav['label'] = "수도권": 1, if voice_wav['label'] = "경상": 2, if voice_wav['label'] = "전라": 3, if voice_wav['label'] = "충청": 4
#voice_wav[(voice_wav['code'] == '수도권')] = '1'

In [8]:
#apply함수로 code내 데이터 변경
def loc_code(region):
    if region == '수도권':
        return 0
    elif region == '경상':
        return 1
    elif region == '전라':
        return 2
    elif region == '충청':
        return 3
    elif region == '제주':
        return 4
    
voice_wav['code']= voice_wav['code'].apply(loc_code)
voice_wav
    

,data,label,code
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",수도권,0
2,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
3,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
4,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
...,...,...,...
7952,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
7953,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
7954,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2
7955,"[6.1035156e-05, 3.0517578e-05, 3.0517578e-05, ...",전라,2


In [9]:
voice_wav.head()

,data,label,code
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",수도권,0
2,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
3,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
4,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0


In [10]:
voice_wav['code'].unique() #코드에 들어간 종류

array([0, 2, 4, 3, 1], dtype=int64)

In [11]:
#code컬럼으로 오름차순으로 정렬
voice_wav.sort_values(by='code',axis=0,ascending= True,inplace=True)

In [12]:
voice_wav.head()

,data,label,code
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",수도권,0
6622,"[-0.0026550293, -0.0026855469, -0.0022277832, ...",수도권,0
6623,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",수도권,0
6624,"[-0.0026550293, -0.0026855469, -0.0022277832, ...",수도권,0
6625,"[-0.0026550293, -0.0026855469, -0.0022277832, ...",수도권,0


##### 전체대사 스크립트

In [15]:
#대사수집 예시
import json
with open('./el_voice/노인남여_노인대화52_F_som020508_79_충청_실내_57254.json', encoding='utf-8') as t:
    script= json.load(t)

print(script['발화정보']['stt'])

작년에 좀 이상한 게 와서 그런데 지금은 많이 완쾌된 상태예요



In [149]:
#전체대사 리스트
import re

scpt_folder= ('./el_voice/')
scpt_lst= []
for file in tqdm(os.listdir(scpt_folder),colour='green'): #tqdm은 진행progress보여주는 함수, os는 디렉토리불러오기
        if 'json' in file:
            scrpt_file_path= os.path.join(scpt_folder,file) #os.path.join 폴더와 파일 경로 합치기
            
            with open(scrpt_file_path, encoding='utf-8') as t: #json파일 불러오기
                script= json.load(t) #변수에 저장
                txt=(script['발화정보']['stt'].replace('\r\n','')) #대사만 추출하고 끝에 문자 삭제
                rmve_bracket = "\(.*\)|\s-\s.*"  #괄호와 괄호안의 정보
                txt_1=re.sub(rmve_bracket,'',txt) #(변경 전/ 변경 후/ 적용할 텍스트)
            scpt_lst.append(txt_1) #변형된 대사들 리스트 저장

len(scpt_lst)


100%|██████████| 13938/13938 [00:11<00:00, 1227.92it/s]


6969

In [150]:
len(scpt_lst)

6969

In [19]:
#전체대사 한줄로 만들기
#scpt_uni_lst=' '.join(scpt_lst)

##### 문장의 유사도

##### countvectorizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
#sklearn이 제공하는 문장을 벡터로 변환하는 함수 CountVectorizer
vectorizer= CountVectorizer(min_df=1)
#거리를 구하는 것이어서 훈련용 문장들을
#지도할 내용이 없다

In [151]:
#전체 대사 형태소분석
from konlpy.tag import Okt
hts=Okt() #한글 형태소 분석
morphs_lst=[hts.morphs(row) for row in scpt_lst]
#대사 리스트에서 한문장씩 꺼내와 형태소로 분석
morphs_lst


[['블라디보스토크',
  '그',
  '쪽',
  '가보신',
  '분',
  '들',
  '은',
  '우리나라',
  '보다',
  '좀',
  '많이',
  '뒤',
  '처진',
  '느낌',
  '이래'],
 ['좀', '어렵게', '사시', '는', '분들', '많고', '이래서', '많이', '안타까웠다고', '그', '러더', '라고'],
 ['우리',
  '가',
  '저번',
  '에',
  '이스라엘',
  '갈',
  '때',
  '잠깐',
  '경유',
  '하면서',
  '겐트',
  '를',
  '들렀잖아요'],
 ['거기',
  '사람',
  '들',
  '보',
  '니까',
  '아직',
  '우리나라',
  '보다는',
  '발전',
  '이',
  '덜',
  '됐구나',
  '싶었어요'],
 ['저',
  '는',
  '그런',
  '곳',
  '에',
  '정말',
  '가고',
  '싶은',
  '곳',
  '을',
  '꼽자',
  '면',
  '사실',
  '아프리카',
  '쪽',
  '이에요'],
 ['자연',
  '이',
  '아직',
  '훼손',
  '되지',
  '않은',
  '것',
  '과',
  '또',
  '순수한',
  '사람',
  '들',
  '이',
  '있잖아요'],
 ['가기',
  '는',
  '어렵지만',
  '그',
  '곳',
  '의',
  '자연',
  '에서',
  '동물',
  '식물',
  '같은',
  '걸',
  '느낄',
  '수',
  '있잖아요'],
 ['살면서',
  '다시',
  '보지',
  '못',
  '할',
  '것',
  '들',
  '을',
  '아프',
  '리',
  '에선',
  '보고',
  '느끼고',
  '할',
  '수',
  '있다',
  '하더라고'],
 ['시간',
  '이',
  '되면',
  '정말',
  '한번',
  '가고',
  '싶어요',
  '남쪽',
  '으로',
  '남아공',
  '쪽',
  '도',
  '좋을'

In [152]:
print(morphs_lst[45:48])

[['우리', '아는', '분', '의', '딸', '보', '니까', '신혼여행', '으로', '하와이', '도', '많이', '가더라고요'], ['그리고', '필리핀', '그', '어디', '지', '괌', '그', '필리핀', '에', '휴양지', '뭐', '죠', '바닷가', '유명한데'], ['그', '쪽', '으로도', '많이', '들', '가고', '또', '유럽', '도', '많이', '가더라고요', '요새', '는']]


In [153]:
#나중에 nltk를 쓸건데 보통 영어로 nltk를 쓰면 띄어쓰기로 문장안의 단어들을 구분함
#그렇기 때문에 우리는 한글로 된 문장을 형태소 단위로 띄어쓰기 해줘서
#이 리스트형 데이터를 스트링으로 바꿔줄것이다.

contents_for_vectorize= [] 

for content in morphs_lst: #컨텐츠 토큰에서 하나씩 가져오는데
    sentence= '' #먼저 한 문장씩 아무것도 없는 null string으로 가져온다
    for word in content: #한 단어씩 들어오면 
        sentence= sentence + ' ' + word 
        #아무것도 없는 sentence에  한칸띄우고 단어를 더해라
    contents_for_vectorize.append(sentence)
    #위의 결과로 만들어지는 sentence를 append시켜라

#형태소단위로 띄워쓰기된 전체대사 문장들
contents_for_vectorize

[' 블라디보스토크 그 쪽 가보신 분 들 은 우리나라 보다 좀 많이 뒤 처진 느낌 이래',
 ' 좀 어렵게 사시 는 분들 많고 이래서 많이 안타까웠다고 그 러더 라고',
 ' 우리 가 저번 에 이스라엘 갈 때 잠깐 경유 하면서 겐트 를 들렀잖아요',
 ' 거기 사람 들 보 니까 아직 우리나라 보다는 발전 이 덜 됐구나 싶었어요',
 ' 저 는 그런 곳 에 정말 가고 싶은 곳 을 꼽자 면 사실 아프리카 쪽 이에요',
 ' 자연 이 아직 훼손 되지 않은 것 과 또 순수한 사람 들 이 있잖아요',
 ' 가기 는 어렵지만 그 곳 의 자연 에서 동물 식물 같은 걸 느낄 수 있잖아요',
 ' 살면서 다시 보지 못 할 것 들 을 아프 리 에선 보고 느끼고 할 수 있다 하더라고',
 ' 시간 이 되면 정말 한번 가고 싶어요 남쪽 으로 남아공 쪽 도 좋을 것 같아',
 ' 나중 에 때 가 되면 우리 같이 돈 을 모아서 한번 가보기로 해 요',
 ' 근데 아프리카 쪽 으로는 시간 이 진짜 좀 많이 필요한 것 같아요',
 ' 가는데도 한 이삼일 정도 걸리고 오는 데도 이삼일 걸릴거니까 요',
 ' 또 시간 을 많이 들여가지고 가야 되는 이러한 곳 이고 하니 까요',
 ' 나 는 러시아 괜찮다고 생각 하는데 가면 우리 가 꼭 한 군데 만 보는 게 아니잖아',
 ' 그 사람 들 생활 방식 이 라든가 걸 배울 수 있지',
 ' 그래서 배 타고 도 많이 가더라고요 부산 에서 출발 하는 배도 있다던데',
 ' 물론 그냥 비행기 를 타고 우리 가 시간 맞춰 봐서 가면 되 기도 하지',
 ' 비행기 타고 그냥 안 힘들게 짧은 시간 에 갔다가 오는 것 도 괜찮고',
 ' 아니면 우리 가 동남아 웬만한 곳 을 우리 가 다 갔다 왔잖아',
 ' 그런 데 는 더 가고 싶은 없으면 일본도 괜찮을 것 같아',
 ' 일본 같은 이니까',
 ' 그래서 또 괜찮은 것 같아 우리 가 이런',
 ' 이렇게 좀 의미 있게 잘 보내는 것 도 우리 한테 는 좀 소중한 거 잖아',
 ' 주어진 시간 이 한정 돼 있고 

In [154]:
#전체대사 벡터화
vectorizer=CountVectorizer(min_df=1)
X= vectorizer.fit_transform(contents_for_vectorize) 
num_sample, num_features = X.shape
num_sample, num_features
print(f'X에는 {num_sample}개의 문장에 말뭉치 단어가 총 {num_features}개 존재')

X에는 6969개의 문장에 말뭉치 단어가 총 5397개 존재


In [23]:
X.toarray().transpose()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [155]:
#인풋음성데이터 처리
import speech_recognition as sr #stt


test_folder="./voice/"
dir_list=os.listdir(test_folder)

best_dist=[]
text_data=[]
best_dist_post=[]


for test_voice in dir_list:

    test_file= os.path.join(test_folder,test_voice)
    r= sr.Recognizer()

    #stt
    with sr.AudioFile(test_file) as source:
        audio=r.record(source,duration=120)
    stt_voice= r.recognize_google(audio_data=audio, language='ko-KR')
    stt_voice_tokens= [stt_voice] #텍스트화시킨 문장 리스트에 저장
    
    
      

    # 인풋 데이터 벡터화
    stt_voice_veced=vectorizer.transform(word_tokens(stt_voice_tokens))
    #word_tokens함수화 시켜서 형태소단위의 문장 완성
    stt_voice_veced.toarray()
            
   

    #거리계산
    dist= [dist_norm(each,stt_voice_veced) for each in X]
    
    
    #오류
    # stt_voice_tokenizer= hts.morphs(stt_voice)
    # stt_voice_tokens.append(stt_voice_tokenizer) -여기서 2중 리스트가 되어버림         
 

    # 유사도 측정
    print('Best post is', dist.index(min(dist)), ', dist= ', min(dist))
#min즉 거리가 가장 짧은 애는 몇번째 애냐 (0,1,2,3)이고 그 값은 2.0이다
    print('Test post is --> ', stt_voice)
    print('Best dist post is --> ', scpt_lst[dist.index(min(dist))])
    best_dist.append(min(dist))
    text_data.append(stt_voice)
    best_dist_post.append(scpt_lst[dist.index(min(dist))])
    
    


result2:
{   'alternative': [   {   'confidence': 0.86375237,
                           'transcript': '우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 '
                                         '기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 서울에 만난 것 같은 기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 겨울에 만난 것 같은 기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 서울에 맛난 것 같은 기분이야'},
                       {   'transcript': '우리가 만난지 몇 년 되지는 않았지만 더 오래 맛난 것 같은 '
                                         '기분이야'}],
    'final': True}


c:\Users\hhjoo\anaconda3\envs\nltk_setting\lib\site-packages\scipy\sparse\_base.py:651: RuntimeWarning: divide by zero encountered in double_scalars
  return self.astype(np.float_)._mul_scalar(1./other)


Best post is 3966 , dist=  0.0
Test post is -->  우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야
Best dist post is -->  우리가 만난 지 몇 년 되지는 않았지만  더 오래 만난 것 같은 기분이야
result2:
{   'alternative': [   {   'confidence': 0.76711464,
                           'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 '
                                         '안대'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰지도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰기도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰기도 안대'}],
    'final': True}
Best post is 4455 , dist=  0.4714045207910317
Test post is -->  꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안대
Best dist post is -->  꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고  약초로 쓰기도 한대
result2:
{   'alternative': [   {   'confidence': 0.78437066,
                           'transcript': '그리고 더 깊이 더 빠를 것 같아서요 기대 돼요 꼭 가보고 싶은 '
          

In [27]:
X.shape

(3019, 3368)

In [117]:
stt_voice_veced.shape #여기가 1로 나왔어야했다 #이중리스트여서 그랬다

(4, 3368)

In [156]:
#input 데이터 결과 데이터프레임으로 만들기 
import pandas as pd
data={
    'Best_dist':best_dist,
    'Input_Data':text_data,
    'Best_Post':best_dist_post
}
test=pd.DataFrame(data)
test.head()

,Best_dist,Input_Data,Best_Post
0,0.000000,우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야,우리가 만난 지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야
1,0.471405,꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안대,꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고 약초로 쓰기도 한대
2,0.698672,그리고 더 깊이 더 빠를 것 같아서요 기대 돼요 꼭 가보고 싶은 곳이네요,그리고 더 깊이도 깊을 것 같아서 기대돼요 꼭 가보고 싶은 곳이에요


In [ ]:
#오류났던 부분
#형태소단위의 문장으로 만들면서 리스트가 이중으로 생겨버림
#그래서 dist구하는데서 shape가 다르다고 나옴

# for voices in stt_voice_tokens:
#             sentence='' 
#             for word in voices:
#                 sentence= sentence+ ' ' + word
#             stt_voice_for_vec.append(sentence)

#     # 인풋 데이터 벡터화
#     stt_voice_veced=vectorizer.transform(stt_voice_for_vec)
#     stt_voice_veced.toarray()

#     dist= [dist_raw(each,stt_voice) for each in X]

# #원본
#     # for tvv in stt_voice_veced:
#     #     dist= [dist_raw(each,tvv) for each in X]
#     print('Best post is', dist.index(min(dist)), ', dist= ', min(dist))
# #min즉 거리가 가장 짧은 애는 몇번째 애냐 (0,1,2,3)이고 그 값은 2.0이다
#     print('Test post is --> ', stt_voice)
#     print('Best dist post is --> ', scpt_lst[dist.index(min(dist))])

In [60]:
#인풋데이터 음성으로 출력
from gtts import gTTS
from IPython.display import Audio
for index, row in test.iterrows():
    print('표준데이터: ', row['Best_Post'])
    tts= gTTS(text=row['Best_Post'], lang='ko',slow=True)
    file=f'./tts_file/음성변환{index}.wav'
    tts.save(file)  
    display(Audio(file,autoplay=True))
    

표준데이터:  우리가 만난 지 몇 년 되지는 않았지만  더 오래 만난 것 같은 기분이야


표준데이터:  꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고  약초로 쓰기도 한대


표준데이터:  그리고 더 깊이도 깊을 것 같아서 기대돼요 꼭 가보고 싶은 곳이에요


##### TF-IDF

In [157]:
#sklearn이 제공하는 문장을 벡터로 변환하는 함수
from sklearn.feature_extraction.text import TfidfVectorizer
#최소활성화빈도
vectorizer_tf= TfidfVectorizer(min_df=1, decode_error='ignore')
X_1= vectorizer_tf.fit_transform(contents_for_vectorize)
num_samples,num_features=X_1.shape
X_1.toarray().transpose()
# 가중치와 역 가중치가 반영되면서 배열의 값이 바뀜
print(f'X에는 {num_samples}개의 문장에 말뭉치 단어가 총 {num_features}개 존재')


X에는 6969개의 문장에 말뭉치 단어가 총 5397개 존재


In [158]:
#인풋음성데이터 처리
test_folder="./voice/"
dir_list=os.listdir(test_folder)

best_dist_tf=[]
text_data_tf=[]
best_dist_post_tf=[]


for test_voice in dir_list:
    test_file= os.path.join(test_folder,test_voice)
    r= sr.Recognizer()

    #stt
    with sr.AudioFile(test_file) as source:
        audio=r.record(source,duration=120)
    stt_voice= r.recognize_google(audio_data=audio, language='ko-KR')
    stt_voice_tokens= [stt_voice] #텍스트화시킨 문장 리스트에 저장
    
    
      

    # 인풋 데이터 벡터화
    stt_voice_veced=vectorizer_tf.transform(word_tokens(stt_voice_tokens))
    #word_tokens함수화 시켜서 형태소단위의 문장 완성
    stt_voice_veced.toarray()
            
   

    #거리계산
    dist= [dist_norm(each,stt_voice_veced) for each in X_1]
    
    
    #오류
    # stt_voice_tokenizer= hts.morphs(stt_voice)
    # stt_voice_tokens.append(stt_voice_tokenizer) -여기서 2중 리스트가 되어버림         
 

    # 유사도 측정
    print('Best post is', dist.index(min(dist)), ', dist= ', min(dist))
#min즉 거리가 가장 짧은 애는 몇번째 애냐 (0,1,2,3)이고 그 값은 2.0이다
    print('Test post is --> ', stt_voice)
    print('Best dist post is --> ', scpt_lst[dist.index(min(dist))])
    best_dist_tf.append(min(dist))
    text_data_tf.append(stt_voice)
    best_dist_post_tf.append(scpt_lst[dist.index(min(dist))])

result2:
{   'alternative': [   {   'confidence': 0.86375219,
                           'transcript': '우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 '
                                         '기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 서울에 만난 것 같은 기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 겨울에 만난 것 같은 기분이야'},
                       {'transcript': '우리가 만난지 몇 년 되지는 않았지만 서울에 맛난 것 같은 기분이야'},
                       {   'transcript': '우리가 만난지 몇 년 되지는 않았지만 더 오래 맛난 것 같은 '
                                         '기분이야'}],
    'final': True}


c:\Users\hhjoo\anaconda3\envs\nltk_setting\lib\site-packages\scipy\sparse\_base.py:651: RuntimeWarning: divide by zero encountered in double_scalars
  return self.astype(np.float_)._mul_scalar(1./other)


Best post is 3966 , dist=  0.0
Test post is -->  우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야
Best dist post is -->  우리가 만난 지 몇 년 되지는 않았지만  더 오래 만난 것 같은 기분이야
result2:
{   'alternative': [   {   'confidence': 0.76711476,
                           'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 '
                                         '안대'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰지도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰기도 안 돼'},
                       {'transcript': '꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초로 쓰기도 안대'}],
    'final': True}
Best post is 4455 , dist=  0.5195190732460624
Test post is -->  꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안대
Best dist post is -->  꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고  약초로 쓰기도 한대
result2:
{   'alternative': [   {   'confidence': 0.78437054,
                           'transcript': '그리고 더 깊이 더 빠를 것 같아서요 기대 돼요 꼭 가보고 싶은 '
          

In [159]:
#변환결과 데이터프레임화
data_tf={
    'Best_dist':best_dist_tf,
    'Input_Data':text_data_tf,
    'Best_Post':best_dist_post_tf
}
test_tf=pd.DataFrame(data_tf)
test_tf.head()

,Best_dist,Input_Data,Best_Post
0,0.000000,우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야,우리가 만난 지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야
1,0.519519,꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안대,꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고 약초로 쓰기도 한대
2,0.756538,그리고 더 깊이 더 빠를 것 같아서요 기대 돼요 꼭 가보고 싶은 곳이네요,그리고 더 깊이도 깊을 것 같아서 기대돼요 꼭 가보고 싶은 곳이에요


In [160]:
test.head()

,Best_dist,Input_Data,Best_Post
0,0.000000,우리가 만난지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야,우리가 만난 지 몇 년 되지는 않았지만 더 오래 만난 것 같은 기분이야
1,0.471405,꽃송이처럼 보이는 겨우살이를 사다가 팔기도 하고 약초를 쓰기도 안대,꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고 약초로 쓰기도 한대
2,0.698672,그리고 더 깊이 더 빠를 것 같아서요 기대 돼요 꼭 가보고 싶은 곳이네요,그리고 더 깊이도 깊을 것 같아서 기대돼요 꼭 가보고 싶은 곳이에요


In [65]:
#인풋데이터 음성으로 변환
for index,row in test_tf.iterrows():
    print('표준어 데이터 : ',row['Best_Post'])
    tts=gTTS(text=row['Best_Post'], lang='ko', slow=True)
    file_name=f"./tts_file/음성테스트{index}_tf.wav"
    tts.save(file_name)
    display(Audio(file_name, autoplay=True))

표준어 데이터 :  우리가 만난 지 몇 년 되지는 않았지만  더 오래 만난 것 같은 기분이야


표준어 데이터 :  꽃송이처럼 보이는 겨우살이를 따다가 팔기도 하고  약초로 쓰기도 한대


표준어 데이터 :  그리고 더 깊이도 깊을 것 같아서 기대돼요 꼭 가보고 싶은 곳이에요


##### 특징 추출하기

In [ ]:
# 정확도 평가
def ACCURACY(true,pred):
    score=np.mean(true==pred)
    return score

In [13]:
#특징 추출 함수
def preprocess_train_dataset(data):
    mfcc_features=[]
    melspectrogram_features=[]
    spec_central_features=[]
    
    for i in data:
        #MFCC
        mfcc= librosa.feature.mfcc(y=i,
                                        sr=16000,
                                        n_fft=2048,
                                        hop_length=512,
                                        n_mfcc=40)
        mfccs=np.mean(mfcc.T,axis=0)
        mfcc_features.append(mfccs)
    
        #Mel-Spectrogram
        mel_spec= librosa.feature.melspectrogram(y=i,
                                        sr=16000, 
                                        n_fft=2048,
                                        hop_length=512,
                                        n_mels=40)
        melspectrogram=np.mean(mel_spec.T,axis=0)
        melspectrogram_features.append(melspectrogram)                                 

        #Spectral_Centroid 소리의 무게중심
        spec_cent= librosa.feature.spectral_centroid(y=i,
                                        sr=16000,
                                        n_fft=2048, 
                                        hop_length=512)
        spec_cents=np.mean(spec_cent.T,axis=0)
        spec_central_features.append(spec_cents)
    
    return np.array(mfcc_features), np.array(melspectrogram_features), np.array(spec_central_features)



In [14]:
#train data넣어서 특징 추출 후 각 특성에 벡터 저장

#라벨값 제거
voice_wav_label_removed = voice_wav.drop(['code','label'], axis=1)

mfcc_features,melspectrogram_features,spec_central_features=preprocess_train_dataset(voice_wav_label_removed.data)

all_features=[]
all_features.append(mfcc_features)
all_features.append(melspectrogram_features)
all_features.append(spec_central_features)
aspect_names=['mel_spectogram','MFCC','Spectral_Centroid']

In [15]:
Classified_data=[]
results=[]
models=[]
models.append(('RandomForestClassifier', RandomForestClassifier(max_depth=4)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(max_depth=2)))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('XgboostClassifier', XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)))

#훈련데이터에서 정답 라벨분리
#X_voice = voice_wav.drop(['code','label'] axis=1)
y_voice= voice_wav['code']

for n in range(0,len(all_features)):
    #학습/ 검증 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(all_features[n],y_voice,test_size=0.4)
    print('train_set: ', X_train.shape, y_train.shape)
    print('test_set: ', X_test.shape, y_test.shape)

    #모델 성능확인
    for name,model in models:
       
        model.fit(X_train,y_train)
        pred=model.predict(X_test)
        score=accuracy_score(y_test,pred)
        Classified_data.append([aspect_names[n],name,round(score*100,3)])
        print(f'{name }모델의 정확도는 {score*100:.3f}% 입니다.')


train_set:  (4774, 40) (4774,)
test_set:  (3183, 40) (3183,)
RandomForestClassifier모델의 정확도는 97.204% 입니다.
DecisionTreeClassifier모델의 정확도는 68.834% 입니다.
AdaBoostClassifier모델의 정확도는 69.274% 입니다.
GradientBoostingClassifier모델의 정확도는 99.623% 입니다.
XgboostClassifier모델의 정확도는 99.717% 입니다.
train_set:  (4774, 40) (4774,)
test_set:  (3183, 40) (3183,)
RandomForestClassifier모델의 정확도는 90.104% 입니다.
DecisionTreeClassifier모델의 정확도는 58.058% 입니다.
AdaBoostClassifier모델의 정확도는 66.070% 입니다.
GradientBoostingClassifier모델의 정확도는 98.586% 입니다.
XgboostClassifier모델의 정확도는 99.152% 입니다.
train_set:  (4774, 1) (4774,)
test_set:  (3183, 1) (3183,)
RandomForestClassifier모델의 정확도는 58.278% 입니다.
DecisionTreeClassifier모델의 정확도는 57.210% 입니다.
AdaBoostClassifier모델의 정확도는 48.571% 입니다.
GradientBoostingClassifier모델의 정확도는 55.954% 입니다.
XgboostClassifier모델의 정확도는 57.273% 입니다.


In [16]:
cal_result=pd.DataFrame(Classified_data,columns=['특징추출 알고리즘','분류모델','정확도(%)'])
cal_result

,특징추출 알고리즘,분류모델,정확도(%)
0,mel_spectogram,RandomForestClassifier,97.204
1,mel_spectogram,DecisionTreeClassifier,68.834
2,mel_spectogram,AdaBoostClassifier,69.274
3,mel_spectogram,GradientBoostingClassifier,99.623
4,mel_spectogram,XgboostClassifier,99.717
5,MFCC,RandomForestClassifier,90.104
6,MFCC,DecisionTreeClassifier,58.058
7,MFCC,AdaBoostClassifier,66.070
8,MFCC,GradientBoostingClassifier,98.586
9,MFCC,XgboostClassifier,99.152


##### 인풋데이터 테스트

In [17]:
test_wav=test_dataset()

100%|██████████| 112/112 [00:00<00:00, 365.26it/s]

dataset완성


In [18]:
test_wav.head()

,data,file_name
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48251.wav
1,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48252.wav
2,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48253.wav
3,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48254.wav
4,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48255.wav


In [19]:
#test data넣어서 특징 추출 후 각 특성에 벡터 저장

mfcc_features,melspectrogram_features,spec_central_features=preprocess_train_dataset(test_wav.data)

all_testf_features=[]
all_testf_features.append(mfcc_features)
all_testf_features.append(melspectrogram_features)
all_testf_features.append(spec_central_features)
aspect_names=['mel_spectogram','MFCC','Spectral_Centroid']

In [20]:
# 트레인셋/검증셋 나눠 성능을 확인했던 것을 하나의 트레인 셋으로 통합하여 다시 모델 생성 후 예측
Classified_data=[]
results=[]
models=[]
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('XgboostClassifier', XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)))
for i in range(0,len(all_features)):    
    for name,clf in models:
        col_name=f'{aspect_names[i]} {name}'
        
        # #모델 학습
        # model=clf
        clf.fit(all_features[i],voice_wav.code)
        #모델 예측
        # prediction=clf.predict(all_testf_features[i])
        # test_wav[col_name]=prediction
        test_wav[col_name]=clf.predict(all_testf_features[i])




In [21]:
test_wav

,data,file_name,mel_spectogram RandomForestClassifier,mel_spectogram DecisionTreeClassifier,mel_spectogram AdaBoostClassifier,mel_spectogram GradientBoostingClassifier,mel_spectogram XgboostClassifier,MFCC RandomForestClassifier,MFCC DecisionTreeClassifier,MFCC AdaBoostClassifier,MFCC GradientBoostingClassifier,MFCC XgboostClassifier,Spectral_Centroid RandomForestClassifier,Spectral_Centroid DecisionTreeClassifier,Spectral_Centroid AdaBoostClassifier,Spectral_Centroid GradientBoostingClassifier,Spectral_Centroid XgboostClassifier
0,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48251.wav,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0
1,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48252.wav,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48253.wav,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
3,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48254.wav,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48255.wav,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48256.wav,0,0,3,0,0,0,0,0,0,0,1,1,1,1,1
6,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48257.wav,0,0,0,0,0,0,0,0,0,0,3,3,1,0,0
7,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48258.wav,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48259.wav,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,"[0.00012207031, 0.0005187988, 0.0007324219, 0....",노인남여_노인대화43_F_5285b_60_수도권_실내_48260.wav,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0


In [22]:
test_wav.to_excel('test_wav3.xlsx')

In [23]:
sido_dict={'수도권':0,'경상':1,'전라':2,'충청':3, '제주':4}


#성능평가
Classified_test_data=[]
models=[]
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('XgboostClassifier', XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)))

# model.fit(all_features[n],train_wav.label)
# pred=model.predict(all_testf_features[n])

# test데이터 label만들기
test_label=[]
for idx, row in test_wav.iterrows():
    if '수도권' in row['file_name']:
        test_label.append(0)
    else:
        f_name=row['file_name']
        f_name=f_name[str(f_name).find('_실')-2:str(f_name).find('_실')]
        test_label.append(sido_dict[f_name])


#모델 성능확인
for n in range(0,len(all_features)):
    for name,model in models:
        model.fit(all_features[n],voice_wav.code)
        pred=model.predict(all_testf_features[n])
        test_score=accuracy_score(test_label,pred)
        Classified_test_data.append([aspect_names[n],name,round(test_score*100,3)])
        print(f'{name}모델의 정확도는 {test_score*100:.3f}% 입니다.')

RandomForestClassifier모델의 정확도는 98.214% 입니다.
DecisionTreeClassifier모델의 정확도는 82.143% 입니다.
AdaBoostClassifier모델의 정확도는 62.500% 입니다.
GradientBoostingClassifier모델의 정확도는 100.000% 입니다.
XgboostClassifier모델의 정확도는 98.214% 입니다.
RandomForestClassifier모델의 정확도는 85.714% 입니다.
DecisionTreeClassifier모델의 정확도는 78.571% 입니다.
AdaBoostClassifier모델의 정확도는 50.000% 입니다.
GradientBoostingClassifier모델의 정확도는 83.929% 입니다.
XgboostClassifier모델의 정확도는 87.500% 입니다.
RandomForestClassifier모델의 정확도는 51.786% 입니다.
DecisionTreeClassifier모델의 정확도는 51.786% 입니다.
AdaBoostClassifier모델의 정확도는 30.357% 입니다.
GradientBoostingClassifier모델의 정확도는 69.643% 입니다.
XgboostClassifier모델의 정확도는 67.857% 입니다.


In [24]:
testdata_cal_result=pd.DataFrame(Classified_test_data,columns=['특징추출 알고리즘','분류모델','정확도(%)'])
testdata_cal_result

,특징추출 알고리즘,분류모델,정확도(%)
0,mel_spectogram,RandomForestClassifier,98.214
1,mel_spectogram,DecisionTreeClassifier,82.143
2,mel_spectogram,AdaBoostClassifier,62.500
3,mel_spectogram,GradientBoostingClassifier,100.000
4,mel_spectogram,XgboostClassifier,98.214
5,MFCC,RandomForestClassifier,85.714
6,MFCC,DecisionTreeClassifier,78.571
7,MFCC,AdaBoostClassifier,50.000
8,MFCC,GradientBoostingClassifier,83.929
9,MFCC,XgboostClassifier,87.500
